Import Libraries

In [1]:
from distilbert_model_fintuned import DistilBERTClass,Classifier
import procesing_pdf as ppdf
import pandas as pd
import os
import numpy as np
import locations_model as lm
import extractkeywords as ek
import re
import citations_model as cm
import time
import cleankeywords as ck


2024-08-11 19:54:46,527 - INFO - intel_extension_for_pytorch auto imported
2024-08-11 19:54:52,921 - INFO - Use pytorch device_name: cpu
2024-08-11 19:54:52,922 - INFO - Load pretrained SentenceTransformer: BAAI/bge-large-en-v1.5
2024-08-11 19:54:55,919 - INFO - Use pytorch device_name: cpu
2024-08-11 19:54:55,920 - INFO - Load pretrained SentenceTransformer: all-MiniLM-L6-v2


learning technology;education technology;instruction technologyaided;edtech innovations;access educational;education tools;literacy software


Import and define models

In [2]:
classifier= Classifier()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.


Auxiliary Functions

In [2]:
categories = {
    0: "Business, Firms, and Finance",
    1: "Conflict, Peace, and Security",
    2: "Education and Human Development",
    3: "Climate and Energy",
    4: "Gender and Inclusion",
    5: "Health and Wellbeing",
    6: "Culture, Institutions, and History",
    7: "Labor and Urban Economics",
    8: "Governance, Political Economy, and Public Management",
    9: "Social Welfare and Public Finance",
    10: "Technology and Data Science",
    11: "Trade, Growth, and Regional Economics"
}
def replace_colons_with_underscores(text):
    invalid_characters = r'[<>:"/\\|?*]'
    return re.sub(invalid_characters, '_', text)
def predict_labels(text):
    exceptions=""
    predictions=classifier.predict(text)
    indices = np.where((predictions >= 0.1) & (predictions <= 0.5))[0]
    predictions=np.where(predictions >= 0.1, 1, 0)
    if len(indices) >= 0:
       exceptions+=";".join([categories[i] for i in indices])
    return predictions, exceptions
def contains_working(text):
    pattern = r'\bworking\b|\bWorking\b'
    if re.search(pattern, text):
        return True
    else:
        return False

In [2]:
import download_pdf_document as dpdf
import pandas as pd
import os
import re
download_dir="d:\proyectos\scraping\downloaded_documents2"
archives = set(os.listdir(download_dir))
def verify_links(row):
    title=row['Title']
    title=title + '.pdf'
    title=replace_colons_with_underscores(title)
    if title in archives:
        return
    else:
       print(title)
       if pd.isna(row['External Link (URL)']):
           dpdf.download_raw_text_from_title(row['Title'])
       else:
           dpdf.donwload_raw_text(row['Title'], row['External Link (URL)'])
def donwload_pdfs():
    dpdf.pass_captcha()
    df=pd.read_excel('data2.xlsx',index_col=False)
    df[['Title', 'External Link (URL)']].apply(verify_links, axis=1)
donwload_pdfs()


What Lebanon_s Elections Can Teach Us About the Importance of Religion in Elections.pdf
Trying on link site......
Could not found any download link on the page
Trying on Harvard site......
Could not find the document on Harvard Site
Trying on LibGen site......
Could not find the document on LibGen Title
Trying on Google Schoolar......
Could not find the document on Google Schoolar
Trying to use abstract as text......
Could not find abstract on the page
Scraping is done
Opportunities for learning in low- and middle-income countries_ A landscape analysis.pdf
Trying on link site......
Could not found download link on the page
Trying on Harvard site......
Could not find the document on Harvard Site
Trying on LibGen site......
WebElement.click() takes 1 positional argument but 2 were given
Trying on Google Schoolar......
Could not find the document on Google Schoolar
Trying to use abstract as text......
Could not find abstract on the page
Scraping is done
Changes in classroom quality predic

In [4]:
def start_scraping():
    download_dir="d:\proyectos\scraping\downloaded_documents2"
    df=pd.read_excel('data2.xlsx',index_col=False)
    for index, row in df.iterrows():
        pdf_filename = f"{row['Title']}.pdf"
        pdf_filename=replace_colons_with_underscores(pdf_filename)
        pdf_path = os.path.join(download_dir, pdf_filename)
        if os.path.exists(pdf_path):
            print("procesando: ",index)
            print(row['Title'])
            #Extract text from pdf
            text_for_keywords,text_for_labels=ppdf.get_clean_text(replace_colons_with_underscores(row['Title']))
            #Predict labels for the text
            labels, label_exceptions = predict_labels(text_for_labels)
            #Update the dataframe with the labels and exceptions
            df.iloc[index, 19:31] = ['Yes' if value == 1 else '' for value in labels]
            #Extract regions and countries from the text
            regions, countries,locations_comments = lm.get_locations(text_for_labels)
            if regions==[0,0,0,0,0,0,0]:
                locations_comments='locations_empty'
                df.at[index,"NOT Int'l Dev?"]='Posible'
            df.iloc[index, 31:38] = ['Yes' if value == 1 else '' for value in regions]
            df.iloc[index,39:40]=countries
            #Extract keywords from the text
            #df.iloc[index,38:39]=ek.extract_keywords(text_for_keywords)
            #Extract citations from the text
            if contains_working(row['Type']):
                if pd.isna(row['External Link (URL)']):
                   df.at[index,'Citation is correct?']='No'
                else:
                   citation=cm.verify_citations(row['External Link (URL)'],row['Citation'],row['Title'])
                   if citation=='no':
                      df.at[index,'Citation is correct?']='No'
                   else:
                      df.at[index,'Citation is correct?']='Yes'
            else:
                    df.at[index, 'Citation is correct?']='Yes'
            df.at[index,'Labels_comments']=label_exceptions
            df.at[index,'Locations_comments']=locations_comments
            
        else:
           df.at[index,'Labels_comments']='not found'
           
    return df
            
    


In [ ]:

download_dir="c:\proyectos\scraping\downloaded_documents"
df=pd.read_excel('new_procesed_data.xlsx',index_col=False)
for index, row in df.iterrows():
    if pd.isna(row['Key Terms (No limit - Separate items by semicolon; do not add a space after the semicolon)']):
        print("procesando: ",index)
        text_for_keywords,text_for_labels=ppdf.get_clean_text(replace_colons_with_underscores(row['Title']))
        keywords=ek.extract_keywords(text_for_keywords)
        df.at[index,'Key Terms (No limit - Separate items by semicolon; do not add a space after the semicolon)']=ck.clean_keywords(keywords)
        #df.at[index,'Key Terms (No limit - Separate items by semicolon; do not add a space after the semicolon)']=ek.extract_keywords(text_for_keywords)
        time.sleep(3)
        df.to_excel('new_procesed_data.xlsx',index=False)
        
        

In [6]:
df=start_scraping()
df.to_excel('new_procesed_data2.xlsx',index=False)

procesando:  0
Implications of Zika virus and congenital Zika syndrome for the number of live births in Brazil
procesando:  2
Managing the Family Firm: Evidence from CEOs at Work
procesando:  3
Male Social Status and Women's Work
procesando:  4
Opportunities for learning in low- and middle-income countries: A landscape analysis
procesando:  5
Nation Building Through Foreign Intervention: Evidence from Discontinuities in Military Strategies
procesando:  6
The Morale Effects of Pay Inequality
procesando:  7
Workplace Signaling and Financial Commitment: Evidence from a Field Experiment
procesando:  8
Imperfect Public Monitoring with a Fear of Signal Distortion
procesando:  9
High-Skilled Migration to the United States and Its Economic Consequences
procesando:  10
High-Skilled Immigration and the Comparative Advantage of Foreign-Born Workers across US Occupations
procesando:  11
Is Populism Necessarily Bad Economics
procesando:  12
Selection and Market Reallocation: Productivity Gains from

In [2]:
download_dir="c:\proyectos\scraping\downloaded_documents"
df=pd.read_excel('new_procesed_data.xlsx',index_col=False)
for index, row in df.iterrows():
    print("procesando: ",index)
    keywords=row['Key Terms (No limit - Separate items by semicolon; do not add a space after the semicolon)']
    clean_keywords=ck.clean_keywords(keywords)
    df.at[index,'Key Terms (No limit - Separate items by semicolon; do not add a space after the semicolon)']=clean_keywords
df.to_excel('final_data.xlsx',index=False)

    

procesando:  0
procesando:  1
procesando:  2
procesando:  3
procesando:  4
procesando:  5
procesando:  6
procesando:  7
procesando:  8
procesando:  9
procesando:  10
procesando:  11
procesando:  12
procesando:  13
procesando:  14
procesando:  15
procesando:  16
procesando:  17
procesando:  18
procesando:  19
procesando:  20
procesando:  21
procesando:  22
procesando:  23
procesando:  24
procesando:  25
procesando:  26
procesando:  27
procesando:  28
procesando:  29
procesando:  30
procesando:  31
procesando:  32
procesando:  33
procesando:  34
procesando:  35
procesando:  36
procesando:  37
procesando:  38
procesando:  39
procesando:  40
procesando:  41
procesando:  42
procesando:  43
procesando:  44
procesando:  45
procesando:  46
procesando:  47
procesando:  48
procesando:  49
procesando:  50
procesando:  51
procesando:  52
procesando:  53
procesando:  54
procesando:  55
procesando:  56
procesando:  57
procesando:  58
procesando:  59
procesando:  60
procesando:  61
procesando:  62
pr

Function to see missing pdf files

In [27]:
import pandas as pd
import os
import re
data=pd.read_excel('data2.xlsx',index_col=False)
download_dir="d:\proyectos\scraping\downloaded_documents2"
archives = set(os.listdir(download_dir))
titles=data['Title'].tolist()
for title in titles:
    name=replace_colons_with_underscores(f"{title}.pdf")
    if name not in archives:
        print(title)

What Lebanon?s Elections Can Teach Us About the Importance of Religion in Elections
The Role of Law in the Production of Inequality: Anthropological and Historical Perspectives
Critical Research in International Law (CRIL): An intensive doctoral retreat
Maternal and Child Health in Uttar Pradesh, India: A Mother?s Story
Working as an ASHA to Improve Maternal and Child Health in Uttar Pradesh, India
Community Health Workers
Which Traffic Policies Work Best for Megacities
How Do China?s New Rich Give Back?
Ant Financial


Function to clean downloaded folder

In [4]:
import os
import re
data=pd.read_excel('data2.xlsx',index_col=False)
download_dir="d:\proyectos\scraping\downloaded_documents2"
archives = set(os.listdir(download_dir))
titles=data['Title'].tolist()
titles_pdf = [title + '.pdf' for title in titles]
titles_pdf = [replace_colons_with_underscores(title) for title in titles_pdf]
for archive in archives:
    if archive not in titles_pdf:
        print(archive)
        os.remove(download_dir + '/' + archive)



Function to save text as pdf

In [3]:
import savetexttopdf as savepdf
import re
text='''Abstract This paper provides a model-based empirical strategy to, (i) detect the presence and gauge the magnitude of government subsidies and (ii) quantify their impact on production reallocation across countries, industry prices, costs and consumer surplus. I construct and estimate an industry model that allows for dynamic agents in both demand and supply and apply my strategy to world shipbuilding, a classic target of industrial policy. I find strong evidence consistent with China having intervened and reducing shipyard costs by 13-20%, corresponding to 1.5 to 4.5 billion US dollars, between 2006 and 2012. The subsidies led to substantial reallocation of ship production across the world, with Japan, in particular, losing significant market share. They also misaligned costs and production, while leading to minor surplus gains for shippers.Keywords: China, government subsidies, industry dynamics, shipbuilding'''
name="asdasdas.pdf"
name=replace_colons_with_underscores(name)
savepdf.save_string_to_pdf(text,name)

PDF file saved successfully.
